# Check customer loyalty

- Goal:
Find customers who can be given discount cards based on their loyalty to any brand sold at our store 

In [102]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# save data to df
df = pd.read_csv('https://stepik.org/media/attachments/lesson/383523/lesson_3_data__1_.csv',
                 encoding='windows-1251')

In [ ]:
df.head()

In [ ]:
# create new df with selected columns
user_df = df[['tc', 'art_sp']]

# rename columns
user_df = user_df.rename(columns={'tc': 'user_id', 'art_sp': 'brand_info'})

In [ ]:
# create new column 'brand_name': save only brand name from brand_info column

user_df['brand_name'] = user_df.brand_info.apply(lambda x: x.split()[-1])

In [ ]:
# find out how many purchases were made per user_id

user_purchases = user_df.groupby('user_id', as_index=False) \
                        .agg({'brand_name':'count'}) \
                        .rename(columns={'brand_name':'purchases'})

In [ ]:
# take a look at stats: how many purchases of the same brand are made on average 
user_purchases.describe()

Half of the people (50%) made 2 purchases of the same brand which doesn't seem too loyal. On the opposite, 25% of customers made 5 and more purchases of the same brand (75% = 5.0 purchases). This sounds a lot more loyal. Let's select only this group of customers to outline customers we want to reward with a discount card.

In [ ]:
# selecting only buyers of 75% percentile of purchases (75% = 5.0 purchases)
user_purchases = user_purchases.query('purchases >= 5')

In [ ]:
# check user loyalty: calculate the share of user favourite product purchases to all user purchases 

favourite_brand_purchases_df = user_df.groupby(['user_id', 'brand_name'], as_index=False) \
        .agg({'brand_info':'count'}) \
        .sort_values(['user_id', 'brand_info'], ascending=[False, False]) \
        .groupby('user_id') \
        .head(1) \
        .rename(columns={'brand_name':'favourite_brand', 'brand_info':'favourite_brand_purchases'})

In [ ]:
favourite_brand_purchases_df.head()

In [ ]:
#check out # of unique brands bought by user
users_unique_brands = user_df.groupby('user_id', as_index=False) \
        .agg({'brand_name': pd.Series.nunique}) \
        .rename(columns={'brand_name':'unique_brands'})

In [ ]:
users_unique_brands.head()

In [ ]:
# merge all three DFs by user_id
loyalty_df = user_purchases \
        .merge(users_unique_brands, on='user_id') \
        .merge(favourite_brand_purchases_df, on='user_id')

In [ ]:
loyalty_df.head()

# Possible answer 1: 
Users loyal to a certain brand are users who made more than 4 purchases (75th percentile of all) of the same brand

In [ ]:
# possible loyal users:
loyal_users = loyalty_df[loyalty_df.unique_brands == 1]

In [ ]:
loyal_users.user_id.count()

Discounts could be given to 911 people who made more than 4 purchases of the same brand

# Possible answer 2:

We could introduce a new metric: 'loyalty_score' (proportion of most often bought brand to all purchases)

In [ ]:
# calculate new metric: 'loyalty_score' 
loyalty_df['loyalty_score'] = loyalty_df.favourite_brand_purchases / loyalty_df.purchases

In [ ]:
# select people who buy 80% of their items from the same brand 
loyal_users_80 = loyalty_df.query('loyalty_score >= 0.8')

In [ ]:
# calculate how many people buy 80% of their items from the same brand 
loyalty_df.query('loyalty_score >= 0.8').user_id.count()

In [ ]:
# visualize loyalty 
ax = sns.displot(loyalty_df.loyalty_score, color='black', kde=False)

We see that most of the customers are 100% loyal to one brand (don't buy any other brands).
911 customers buy only one brand: which they bought 4 times or more. We could give discounts to the most loyal customers only (loyalty score = 1.0). (this corresponds to answer 1).
If we went to give discount cards to a larger number of people, we could also include people with lower loyalty score, i.e.,
0.8, or even 0.7. On average 80% (or 70%) of all purchases they make are from a favourite brand. This could also be considered rather loyal depending on definition.

# We could also take a look at what brands people are more loyal to in general (just because we are curious to know what brand is the most popular one among loyal customers)

In [ ]:
# new df: loyalty_score by brand
brands_loyalty = loyalty_df.groupby('favourite_brand', as_index=False) \
            .agg({'loyalty_score':'median', 'user_id':'count'})

In [ ]:
brands_loyalty

In [ ]:
# vizualise data

plt.figure(figsize=(10, 6.5))
ax = sns.barplot(x='favourite_brand', y='user_id', data=brands_loyalty, palette='BuGn')
sns.despine()

Brand 4 is the most popular one among loyal customers (who make 5 and more purchases). Compared to brand 4, other brands seem to be less popular.

Overall, we could give discounts to 911 customers (they are listed in loyal_users). Their loyalty score is 1.0 which means they only buy from just one brand and bought it more than 4 times. Another option would be to give discounts to people who buy 80% of their items from the same brand. This would make up 1703 customers (loyal_users_80). If we want to make the subset even larger, we could go with 70% loyal to a brand group, etc. Lastly, we see that brand 4 is the most often bought brand by loyal customers. Hence, we could only give discount to this brand if we want.